In [1]:
# import libraries
# %matplotlib notebook
# %matplotlib inline

print('Library versions:')

import keras
print(f'keras:{keras.__version__}')
import pandas as pd
print(f'pandas:{pd.__version__}')
import sklearn
print(f'sklearn:{sklearn.__version__}')
import nltk
print(f'nltk:{nltk.__version__}')
import numpy as np
print(f'numpy:{np.__version__}')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
import demoji
import pickle
import re
import random
import time
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
import tensorflow as tf
print(f'tensofrlow:{tf.__version__}')

import os
os.getcwd()

demoji.__version__
with open(r'../data/Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
    Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}
    
from tqdm.notebook import tqdm as tqdm
from tqdm.notebook import tqdm
tqdm.pandas()

Library versions:
keras:2.8.0
pandas:1.3.4
sklearn:0.24.2
nltk:3.6.7
numpy:1.20.3
tensofrlow:2.8.0


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JiatingChen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import os
os.getcwd()

'C:\\Users\\JiatingChen\\Documents\\nlp-coe\\twitter-conversational-chatbot\\modeling'

## Read Data

In [4]:
""" A kernel posted on Kaggle that shows how to pull just the first consumer request and
    company response from the dataset.
"""

tweets = pd.read_csv('../data/twcs/twcs.csv')


# Pick only inbound tweets that aren't in reply to anything...
first_inbound = tweets[pd.isnull(tweets.in_response_to_tweet_id) & tweets.inbound]
print('Found {} first inbound messages.'.format(len(first_inbound)))

# Merge in all tweets in response
inbounds_and_outbounds = pd.merge(first_inbound, tweets, left_on='tweet_id', 
                                  right_on='in_response_to_tweet_id')
print("Found {} responses.".format(len(inbounds_and_outbounds)))

# Filter out cases where reply tweet isn't from company
inbounds_and_outbounds = inbounds_and_outbounds[inbounds_and_outbounds.inbound_y ^ True]

# Et voila!
print("Found {} responses from companies.".format(len(inbounds_and_outbounds)))
print("Tweets Preview:")
print(inbounds_and_outbounds)

Found 787346 first inbound messages.
Found 875292 responses.
Found 794299 responses from companies.
Tweets Preview:
        tweet_id_x author_id_x  inbound_x                    created_at_x  \
0                8      115712       True  Tue Oct 31 21:45:10 +0000 2017   
1                8      115712       True  Tue Oct 31 21:45:10 +0000 2017   
2                8      115712       True  Tue Oct 31 21:45:10 +0000 2017   
3               18      115713       True  Tue Oct 31 19:56:01 +0000 2017   
4               20      115715       True  Tue Oct 31 22:03:34 +0000 2017   
...            ...         ...        ...                             ...   
875287     2987942      823867       True  Wed Nov 22 07:30:39 +0000 2017   
875288     2987944      823868       True  Wed Nov 22 07:43:36 +0000 2017   
875289     2987946      524544       True  Wed Nov 22 08:25:48 +0000 2017   
875290     2987948      823869       True  Wed Nov 22 08:35:16 +0000 2017   
875291     2987950      823870       

In [5]:
print(f'Data shape: {inbounds_and_outbounds.shape}')

Data shape: (794299, 14)


In [9]:
inbounds_and_outbounds.to_csv('../data/inbounds_and_outbounds.csv', index = False)

## Data Preprocessing

In [ ]:
# Replace anonymized screen names with common token @__sn__
def sn_replace(match):
    _sn = match.group(2).lower()
    if not _sn.isnumeric():
        # This is a company screen name
        return match.group(1) + match.group(2)
    return ' @__sn__'


def replace_ID(text):
    try:
        if isinstance(text, str):
            sn_re = re.compile('(\W@|^@)([a-zA-Z0-9_]+)')
            return sn_re.sub(sn_replace, text)
        else:
            raise ValueError()      
    except ValueError:
        pass
    
def convert_emojis_to_word(text, emoji_dict):
    try:
        if isinstance(text, str):
            for emot in Emoji_Dict:
                text = re.sub(r'(' +emot+ ')', "".join(Emoji_Dict[emot].replace(":"," ")),text)
            return text
        else:
            raise ValueError()      
    except ValueError:
        pass
    

def replace_URL(text):
    try:
        if isinstance(text, str):
            pattern = r"http\S+|https\S+|www\S+"
            return re.sub(pattern, "__URL__", str(text))
        else:
            raise ValueError()      
    except ValueError:
        pass
    

def clean_text(text, emoji_dict = Emoji_Dict):
    try:
        if isinstance(text, str):
            temp1 = replace_ID(text)
            temp2 = convert_emojis_to_word(temp1, emoji_dict)
            temp3 = replace_URL(temp2)
            return temp3   
        else:
            raise ValueError()      
    except ValueError:
        pass

In [6]:
clean_text("testiing case 2: @23456 Please refer to either this link www:/somethinghelpful or this link: http:/somethinghelpful/somethingevermorehelpful. Have a great day😊!")

'testiing case 2: @__sn__ Please refer to either this link __URL__ or this link: __URL__ Have a great day smiling_face_with_smiling_eyes !'

In [16]:
import random
random.seed(1234)
sample = inbounds_and_outbounds.loc[: , ['text_x', 'text_y']].sample(frac = 0.15)

In [19]:
x_text = sample.text_x.progress_apply(lambda txt: clean_text(txt))

  0%|          | 0/119145 [00:00<?, ?it/s]

In [20]:
y_text = sample.text_y.progress_apply(lambda txt: clean_text(txt))

  0%|          | 0/119145 [00:00<?, ?it/s]

In [5]:
df = pd.concat([x_text, y_text], axis = 1)

In [6]:
training_data = df.sample(frac = 0.8, random_state = 1234)
testing_data = df.loc[~df.index.isin(training_data.index.to_list())]

In [8]:
with open('training_data.pickle', 'wb') as handle:
    pickle.dump(training_data, handle)
with open('testing_data.pickle', 'wb') as handle:
    pickle.dump(testing_data, handle)